In [4]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/bersatu-dengan-peradaban-5.mp3
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/ilusi-kuasa-1.mp3
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/janji-tinggal-janji-2.mp3
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/membina-negara-dan-bangsar-bersatu-6.mp3
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/orang-turk-memperjuangkan-hak-mereka-3.mp3
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/audiobook/turki/revolusi-barat-di-timur-4.mp3

In [5]:
import os

os.system(
    'mkdir output-wav-turki output-text-turki'
)
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
import speech_recognition as sr
import soundfile as sf
from tqdm import tqdm

r = sr.Recognizer()

from pydub.silence import split_on_silence, detect_nonsilent
from pydub import AudioSegment
import numpy as np
import malaya_speech
from malaya_speech import Pipeline
from scipy.io.wavfile import write

p = Pipeline()

model_v2 = malaya_speech.vad.deep_model(model = 'vggvox-v2')
pipeline = (
    p.map(malaya_speech.utils.generator.frames, frame_duration_ms = 30)
    .batching(20)
    .foreach_map(model_v2.predict)
    .flatten()
)

In [7]:
from glob import glob

mp3s = glob('*.mp3')
mp3s

['orang-turk-memperjuangkan-hak-mereka-3.mp3',
 'janji-tinggal-janji-2.mp3',
 'membina-negara-dan-bangsar-bersatu-6.mp3',
 'ilusi-kuasa-1.mp3',
 'revolusi-barat-di-timur-4.mp3',
 'bersatu-dengan-peradaban-5.mp3']

In [8]:
for file in mp3s:
    print(file)
    try:
        audio = AudioSegment.from_mp3(file)
        sample_rate = audio.frame_rate
        samples = np.array(audio.get_array_of_samples())
        samples = malaya_speech.astype.int_to_float(samples)
        samples_16k = malaya_speech.resample(samples, sample_rate, 16000)
        frames_16k = list(
            malaya_speech.utils.generator.frames(samples_16k, 30, 16000)
        )
        frames = list(
            malaya_speech.utils.generator.frames(samples, 30, sample_rate)
        )
        result = p.emit(samples_16k)
        frames_deep_v2_batch = [
            (frame, result['flatten'][no]) for no, frame in enumerate(frames)
        ]
        results = malaya_speech.split.split_vad(
            frames_deep_v2_batch, n = 5, negative_threshold = 0.1
        )

        for no in tqdm(range(len(results))):
            result = results[no]

            sf.write('test.wav', result.array, sample_rate)

            try:

                with sr.AudioFile('test.wav') as source:
                    a = r.record(source)

                text = r.recognize_google(a, language = 'ms')
                filename = f'{file}-{no}.wav'
                if len(text):
                    text_filename = (
                        f'output-text-turki/{filename}.txt'
                    )
                    with open(text_filename, 'w') as fopen:
                        fopen.write(text)

                    sf.write(
                        f'output-wav-turki/{filename}',
                        result.array,
                        sample_rate,
                    )
            except Exception as e:
                print(e)
    except Exception as e:
        print(e)

orang-turk-memperjuangkan-hak-mereka-3.mp3


100%|██████████| 502/502 [09:55<00:00,  1.19s/it]


janji-tinggal-janji-2.mp3



100%|██████████| 367/367 [06:32<00:00,  1.07s/it]


membina-negara-dan-bangsar-bersatu-6.mp3


...

100%|██████████| 654/654 [10:53<00:00,  1.00it/s]



revolusi-barat-di-timur-4.mp3


100%|██████████| 612/612 [10:45<00:00,  1.06s/it]


bersatu-dengan-peradaban-5.mp3



100%|██████████| 364/364 [06:52<00:00,  1.13s/it]